In [1]:
from parameters import get_parameters
import os
import re
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
params = get_parameters()

start_year = params["start_year"]
end_year = params["end_year"]
source_curated = params["source_curated"]
source_conformed = params["source_conformed"]

In [3]:
PROJECT_ROOT = Path().resolve().parent
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\dxnin\Documents\dengue-forecast


In [4]:
df_dengue = pd.read_pickle(f"{PROJECT_ROOT}/{source_curated}/dengue.pkl")

In [5]:
climate_variables = [f.replace(".pkl", "") for f in os.listdir(f"{PROJECT_ROOT}/{source_curated}") if f.endswith('.pkl') and f != 'dengue.pkl']
climate_variables = list(set(climate_variables))
climate_variables

['runoff', 'precipitation', 'wind', 'temperature']

In [6]:
climate_data = {}

for var in climate_variables:
    climate_data[var] = pd.read_pickle(f"{PROJECT_ROOT}/{source_curated}/{var}.pkl")

In [7]:
municipios = df_dengue.MUNICIPIO.unique()

for key, df in climate_data.items():
    municipios = np.concatenate((municipios, df.MUNICIPIO.unique()))
municipios = np.unique(municipios)
print("Total of municipios:", len(municipios))
print("List of municipios:\n", municipios)

Total of municipios: 1
List of municipios:
 ['MEDELLÍN']


In [8]:
df_dengue = (
    df_dengue
    .groupby("DATE", as_index=False)
    .agg({"CASES": "sum"})
)

In [9]:
for variable in climate_variables:
    var = variable.lower()

    df = climate_data[var].copy()

    df = df.drop(columns=[c for c in ["GEOMETRY", "LON_LAT"] if c in df.columns], errors="ignore")

    value_vars = [
        col for col in df.columns
        if col not in ["DATE", "ID", "MUNICIPIO"]
    ]

    df = (
        df.melt(
            id_vars=["DATE", "ID", "MUNICIPIO"],
            value_vars=value_vars
        )
        .assign(variable=lambda d: d["variable"] + "_" + d["ID"].astype(str) + "_" + d["MUNICIPIO"])
        .pivot(index="DATE", columns="variable", values="value")
        .reset_index()
    )
    df.columns.name = None
    climate_data[var] = df


In [10]:
for variable in climate_variables:
    var = variable.lower()

    df = climate_data[var]

    # --- TEMPERATURE ---
    if "temperature" in var:
        df = df.apply(lambda x: x - 273.15 if x.name != "DATE" else x)
        print(f"Changed {variable} from K → °to")

    # --- RUNOFFS & PRECIPITATION ---
    elif any(k in var for k in ["runoff", "precipitation"]):
        df = df.apply(lambda x: x * 1000 if x.name != "DATE" else x)
        print(f"Changed {variable} from m → mto")

    # --- WINDS ---
    elif "wind" in var:
        df = df.apply(lambda x: x * 3.6 if x.name != "DATE" else x)
        print(f"Changed {variable} from m/s to km/h")

    else:
        print(f"⚠️ {variable}: no conversion rule found")

    climate_data[var] = df

Changed runoff from m → mto
Changed precipitation from m → mto
Changed wind from m/s to km/h
Changed temperature from K → °to


In [11]:
# Merge all datasets using DATE field
df_total = df_dengue

for key, df in climate_data.items():
    df_total = df_total.merge(df, on=['DATE'], how='left')

In [12]:
pattern = r'^(.*?)_\d+_'
climate_variables_new = sorted(set(re.match(pattern, col).group(1) for col in df_total.columns if re.match(pattern, col)))
climate_variables_new

['PRECIPITATION_MAX', 'RUNOFF_MAX', 'TEMPERATURE_MAX', 'WIND_MAX']

In [13]:
# Check if there are any missing values
df_total[df_total.isnull().any(axis=1)]

,DATE,CASES,RUNOFF_MAX_0_MEDELLÍN,RUNOFF_MAX_1_MEDELLÍN,RUNOFF_MAX_2_MEDELLÍN,RUNOFF_MAX_3_MEDELLÍN,PRECIPITATION_MAX_0_MEDELLÍN,PRECIPITATION_MAX_1_MEDELLÍN,PRECIPITATION_MAX_2_MEDELLÍN,PRECIPITATION_MAX_3_MEDELLÍN,WIND_MAX_0_MEDELLÍN,WIND_MAX_1_MEDELLÍN,WIND_MAX_2_MEDELLÍN,WIND_MAX_3_MEDELLÍN,TEMPERATURE_MAX_0_MEDELLÍN,TEMPERATURE_MAX_1_MEDELLÍN,TEMPERATURE_MAX_2_MEDELLÍN,TEMPERATURE_MAX_3_MEDELLÍN


In [14]:
# Seleccionar valor promedio por variable climática
for var in climate_variables_new:
    cols = [c for c in df_total.columns if c.startswith(var)]   
    col_name = f"{var.split('_')[0]}_AVG_MAX"
    df_total[col_name] = df_total[cols].mean(axis=1)
    df_total.drop(columns=cols, inplace=True)

In [15]:
df_total = df_total.set_index('DATE')
df_total = df_total.asfreq('W')

In [16]:
df_total = df_total.sort_index()
df_total = df_total.iloc[:-1]

In [17]:
# Show dataframe info
print("Total of records:", len(df_total))
print("============================================")
print("Columns and data types:")
print(df_total.dtypes)

Total of records: 886
Columns and data types:
CASES                      int64
PRECIPITATION_AVG_MAX    float64
RUNOFF_AVG_MAX           float64
TEMPERATURE_AVG_MAX      float64
WIND_AVG_MAX             float64
dtype: object


In [18]:
df_total.head()

,CASES,PRECIPITATION_AVG_MAX,RUNOFF_AVG_MAX,TEMPERATURE_AVG_MAX,WIND_AVG_MAX
DATE,,,,,
2007-01-07,0,0.829115,0.057710,21.954080,2.847873
2007-01-14,0,4.017773,0.367180,22.923341,3.972641
2007-01-21,0,8.691750,1.045510,23.034158,3.766555
2007-01-28,0,13.806932,1.639757,24.142465,4.537634
2007-02-04,0,1.075573,0.077687,26.059118,5.366861


In [19]:
pd.to_pickle(df_total, f"{PROJECT_ROOT}/{source_conformed}/data.pkl")